In [1]:
import json
from pathlib import Path

import numpy as np
from tqdm import tqdm

from src.optics.channel_simulation import ChannelSimulator
from src.optics.split_step_fourier import SplitStepFourier

In [3]:
# config
data_len = 50
dir = f'data/{data_len}_samples'
x_path = f'{dir}/data_x.npy'
y_path = f'{dir}/data_y.npy'
conf_path = f'{dir}/conf.json'

In [2]:
cs = ChannelSimulator(m_qam=16,
                      num_symbols=512,
                      normalization_factor=1e-3,
                      dt=1,
                      ssf=SplitStepFourier(
                          b2=-20e-27,
                          gamma=0.003,
                          t0=125e-12,
                          dt=1,
                          z_n=1.51,
                          h=500
                      ),
                      verbose=False)

SSF params: N = 2359, P_0 = 0.00042666666666666667
number of iterations in split step algo: 2359


In [3]:
# generate the date
all_x = np.zeros(shape=(data_len, 512))
all_y = np.zeros(shape=(data_len, 512))
for i in tqdm(range(data_len)):
    x, y = cs.gen_io_data()
    all_x[i], all_y[i] = x, y

  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/1k/xrnsys5s7b39s_ylmkgng4fw0000gn/T/ipykernel_14695/3164716535.py:6: ComplexWarning: Casting complex values to real discards the imaginary part
  all_x[i], all_y[i] = x, y
100%|██████████| 50/50 [01:19<00:00,  1.58s/it]


In [20]:
# save to files
Path(dir).mkdir(parents=True, exist_ok=True)

with open(x_path, 'wb') as f:
    np.save(f, all_x)
    print('saved inputs')
with open(y_path, 'wb') as f:
    np.save(f, all_y)
    print('saved outputs')
with open(conf_path, 'w') as f:
    json.dump(cs.params_to_dict(), f, indent=4)
    print('saved json params')

saved inputs
saved outputs
saved json params


In [4]:
# read
with open(x_path, 'rb') as f:
    all_x_read = np.load(f)
with open(y_path, 'rb') as f:
    all_y_read = np.load(f)
with open(conf_path, 'r') as f:
    conf_read = json.load(f)

In [ ]:
# verify bit exact
assert (all_x_read == all_x).all(), 'oh no the read x is not like the saved x'
assert (all_y_read == all_y).all(), 'oh no the read y is not like the saved y'
assert conf_read == cs.params_to_dict(), 'oh no! the read conf is not like the saved conf'
print('sucess')